# 4E - 14 DAYS (baseline run)

In [4]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns
import json
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.models import load_model
from scipy.stats import zscore

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
## Change directory to where your scripts and data file is placed ### 
BASE_PATH = "./private/thesisFinalFiles/"
os.chdir(BASE_PATH)
print(os.path.abspath('')) 

from preprocessing_exp2 import *
from spacetimemodel import spacetimemodel

In [7]:
data = pd.read_csv('vietnam_dataMatrix_modelling2.csv', header=0)
idx_dic = {}
for col in data.columns:
    idx_dic[col] = data.columns.get_loc(col)
print(idx_dic)
# import sys


{'OID_': 0, 'area': 1, 'landslide': 2, 'su_ID': 3, 'eventDate': 4, 'inventory': 5, 'reliefMax': 6, 'reliefSD': 7, 'planCurMax': 8, 'planCurvSD': 9, 'profCurMax': 10, 'profCurSD': 11, 'slopeMax': 12, 'slopeSD': 13, 'faultDiMax': 14, 'faultDiSD': 15, 'litho': 16, 'landcover': 17, 'F1_max': 18, 'F1_sum': 19, 'F2_max': 20, 'F2_sum': 21, 'F3_max': 22, 'F3_sum': 23, 'F4_max': 24, 'F4_sum': 25, 'F5_max': 26, 'F5_sum': 27, 'F6_max': 28, 'F6_sum': 29, 'F7_max': 30, 'F7_sum': 31, 'F8_max': 32, 'F8_sum': 33, 'F9_max': 34, 'F9_sum': 35, 'F10_max': 36, 'F10_sum': 37, 'F11_max': 38, 'F11_sum': 39, 'F12_max': 40, 'F12_sum': 41, 'F13_max': 42, 'F13_sum': 43, 'F14_max': 44, 'F14_sum': 45, 'F15_max': 46, 'F15_sum': 47, 'F16_max': 48, 'F16_sum': 49, 'F17_max': 50, 'F17_sum': 51, 'F18_max': 52, 'F18_sum': 53, 'F19_max': 54, 'F19_sum': 55, 'F20_max': 56, 'F20_sum': 57, 'F21_max': 58, 'F21_sum': 59, 'F22_max': 60, 'F22_sum': 61, 'F23_max': 62, 'F23_sum': 63, 'F24_max': 64, 'F24_sum': 65, 'F25_max': 66, 'F25

# Preparing rainfall data

## difference

In [8]:
# create a new DataFrame to hold the differences
diffMax_df = pd.DataFrame()
diffSum_df = pd.DataFrame()

df_max = data.iloc[:,np.arange(50,78,2)]
df_sum = data.iloc[:,np.arange(51,78,2)]

# iterate over the columns of the original DataFrame
for i in range(1, len(df_max.columns)):
    if i == 1: 
        diffMax_df[f'diff_{1}'] = 0

    # calculate the difference between this column and the previous column
    diff = df_max.iloc[:, i] - df_max.iloc[:, i-1]
    # add the difference as a new column in the diff_df DataFrame
    diffMax_df[f'diff_{i+1}'] = diff

diffMax_df['diff_1'].fillna(value=0,inplace=True)

for i in range(1, len(df_max.columns)):
    if i == 1: 
        diffSum_df[f'diff_{1}'] = 0

    # calculate the difference between this column and the previous column
    diff = df_sum.iloc[:, i] - df_sum.iloc[:, i-1]
    # add the difference as a new column in the diff_df DataFrame
    diffSum_df[f'diff_{i+1}'] = diff

diffSum_df['diff_1'].fillna(value=0,inplace=True)

# display the resulting DataFrame
diffSum_df.head(10)

diff_1     diff_2     diff_3    diff_4    diff_5     diff_6     diff_7  \
0     0.0   9.710696  16.184493 -3.236900  0.215438 -17.868099   6.540532   
1     0.0   6.144605  10.241009 -2.048202  0.000000 -11.827802   5.151284   
2     0.0  10.557746  17.596243 -3.519248  2.936230 -24.051722   9.780282   
3     0.0   7.531363  11.900925 -4.218611  1.940526 -12.691337   4.967809   
4     0.0  12.447235  19.605534 -7.585746  3.151703 -19.605534   7.085655   
5     0.0   9.887270  16.478782 -3.295755  3.295755 -23.070295  10.147362   
6     0.0   6.512400  10.853999 -2.170801  0.174945 -13.132517   6.057903   
7     0.0   6.507313   3.904389 -2.602926  1.301463  -6.507314   2.755542   
8     0.0   6.983759  11.639598 -2.327920  1.075618 -13.967517   5.718315   
9     0.0   4.594466   7.657443 -1.531489  0.000000  -9.188932   4.311626   

     diff_8     diff_9   diff_10    diff_11    diff_12   diff_13   diff_14  
0  3.848720 -15.394880  0.979931  10.601730   9.434524 -5.588925  2.961902  
1  2.553631 -10.214524  0.255634   6.639711   6.098299 -2.396875  0.772670  
2  4.896277 -15.921331 -1.927152  11.025055   9.234516 -2.575861 -0.362949  
3  2.395039  -8.927871 -1.367964   6.119515   6.849701 -3.851039  2.069589  
4  4.749598 -15.098847 -2.233124  10.065898  10.840843 -5.675763  3.378005  
5  1.360612 -10.574092 -2.114819   8.459274  11.177274 -8.156763  2.718921  
6  2.837765 -11.133694  0.000000   6.849877   6.648347 -1.687277  0.000000  
7  1.786155  -5.358467 -0.893078   3.572312   2.769399  0.000000  0.000000  
8  4.199885 -13.321738  0.000000   6.587186   8.297678 -1.860607  0.000000  
9  1.947706  -7.790820  0.000000   4.869262   4.632146 -1.187675  0.000000

In [9]:
df_cumSumRainfall, df_sumRainfall = cumRainfall(data, 51, 77, 'sum')
df_cumMaxRainfall, df_maxRainfall = cumRainfall(data, 50, 77, 'max')

In [11]:
scaler = MinMaxScaler()

df_sumDiffScaled= scaler.fit_transform(diffSum_df)
df_sumDiffScaled = pd.DataFrame(df_sumDiffScaled, columns=diffSum_df.columns)
df_sumDiffScaled.head(2)

diff_1    diff_2    diff_3    diff_4    diff_5    diff_6    diff_7  \
0     0.0  0.732172  0.773510  0.348107  0.498392  0.158549  0.596939   
1     0.0  0.637948  0.616015  0.379334  0.493465  0.283835  0.559144   

     diff_8    diff_9   diff_10   diff_11   diff_12   diff_13   diff_14  
0  0.432004  0.237318  0.416769  0.866596  0.685095  0.171108  0.533706  
1  0.398977  0.359422  0.379240  0.712965  0.621861  0.250050  0.490652

In [12]:
rainfall_array = rainfallMatrix_4(df_maxRainfall.apply(zscore), df_sumRainfall.apply(zscore), df_cumSumRainfall.apply(zscore), df_sumDiffScaled)

In [15]:
rainfall_array.shape

(2759, 14, 4)

## extracting only the last value of the time series for cumulative sum

In [16]:
# Extracting the last column
last_reading = rainfall_array[:, -1:, -2:-1]
# Extracting the last column and reshaping it
# last_column = my_array[:, -1:, :]
last_reading.shape

(2759, 1, 1)

In [17]:
print(last_reading)

[[[ 3.38156715]]

 [[ 1.74014056]]

 [[ 3.76879337]]

 ...

 [[-0.60817409]]

 [[-0.54537598]]

 [[-0.54054123]]]


In [50]:
print(df_cumSumRainfall.apply(zscore)[:4]) # to check that i indeed use cumsum rainfall

   cumsum_1  cumsum_2  cumsum_3  cumsum_4  cumsum_5  cumsum_6  cumsum_7  \
0  -0.47458  2.151495  4.079327  4.279559  4.269727  4.258325  3.958457   
1  -0.47458  1.090300  2.331823  2.466430  2.452755  2.354627  2.196664   
2  -0.47458  2.403559  4.494410  4.710230  4.863361  4.745165  4.445868   
3  -0.47458  1.502970  2.924331  2.916371  2.946046  2.946346  2.771548   

   cumsum_8  cumsum_9  cumsum_10  cumsum_11  cumsum_12  cumsum_13  cumsum_14  
0  3.798156  3.611587   3.444039   3.409459   3.507406   3.424114   3.381567  
1  2.123340  1.989567   1.854626   1.812410   1.858181   1.800390   1.740141  
2  4.316710  4.202704   3.989519   3.895673   3.923882   3.871753   3.768793  
3  2.666646  2.629023   2.518449   2.438455   2.455456   2.342289   2.263501  


# Processing environmental data

In [21]:
# normalizing continuous static environmental data 
df_staticOnly = data[['litho','landcover',
                    'reliefSD', 'slopeSD', 'cosAspSD', 'sinAspSD', 'faultDiSD',
                     'reliefME', 'plancurME', 'profcurvME', 'slopeME','fautltDiME','sinAspME', 'cosAspME' ]]

catCols = ['landcover','litho']
df_staticOnlyNorm = normCols(df_staticOnly, catCols)
# df_staticOnlyNorm.head(10)

litho          object
landcover      object
reliefSD      float64
slopeSD       float64
cosAspSD      float64
sinAspSD      float64
faultDiSD     float64
reliefME      float64
plancurME     float64
profcurvME    float64
slopeME       float64
fautltDiME    float64
sinAspME      float64
cosAspME      float64
dtype: object 

Index(['reliefSD', 'slopeSD', 'cosAspSD', 'sinAspSD', 'faultDiSD', 'reliefME',
       'plancurME', 'profcurvME', 'slopeME', 'fautltDiME', 'sinAspME',
       'cosAspME'],
      dtype='object')


In [22]:
# one hot encoding the categorical variables in the df 
in_array_env, outDf = catPrep(df_staticOnlyNorm, ['litho','landcover'])

outDf.columns
len(outDf.columns)
env_array = outDf.to_numpy()
env_array.shape

landslide_array = np.array(data['landslide'], dtype=int)
# print(landslide_array)


You have one-hot encoded  2  categorical variables. These are: 

litho 

landcover 



# Data split

In [18]:
#split data first into train and test 
idx=np.arange(0,2759)
idx_tr, idx_ts = train_test_split(idx, test_size=0.25,random_state=42)
idx_tr.sort()
idx_ts.sort()

# Hyperparameterization

## Hyperband optimization

In [56]:
from optimize_model import * 
import keras_tuner as kt

params= json.load(open('Exp4E_oneRainfall.json','r'))
# print(params)

X_train = [env_array[idx_tr],last_reading[idx_tr]]
Y_train = landslide_array[idx_tr]

obj = optimize_model(params['modelparam'])
objfunc = kt.Objective('val_auc', direction='max')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True)
tuner2 = kt.Hyperband(obj, objective=objfunc, 
                       max_epochs=150, factor=3, 
                      project_name='msc-tuning-hyperband-4e-14days-baseline',
                      overwrite=True, seed=10)
tuner2.search(X_train, Y_train, epochs=200, validation_split=0.2, callbacks=[stop_early]) #tf.keras.callbacks.TensorBoard("/tmp/tb_logs")
best_hps = tuner2.get_best_hyperparameters(num_trials=2)[0]

Trial 254 Complete [00h 01m 13s]
val_auc: 0.5682762265205383

Best val_auc So Far: 0.7507935166358948
Total elapsed time: 04h 09m 17s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [28]:
best_hps.values

NameError: name 'best_hps' is not defined

## Bayesian optimization

In [24]:
from optimize_model import * 
import keras_tuner as kt

params= json.load(open('Exp4E_oneRainfall.json','r'))

X_train = [env_array[idx_tr],last_reading[idx_tr]]
Y_train = landslide_array[idx_tr]
objfunc = kt.Objective('val_auc', direction='max')
obj = optimize_model(params['modelparam'])

# note that in older versions, it's obj.build but as of 4/4/23, if build is there, batch size somehow ain't hyperparameterized.
tuner3 = kt.BayesianOptimization(obj, objective=objfunc, 
                      project_name='msc-tuning-bay-4e-14days-baseline', max_trials=50, max_retries_per_trial=5,
                      overwrite=True, seed=10)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True)
tuner3.search(X_train, Y_train, epochs=200, validation_split=0.2, callbacks=[stop_early])
best_hps3 = tuner3.get_best_hyperparameters(num_trials=2)[0]

Trial 50 Complete [00h 00m 49s]
val_auc: 0.6316980719566345

Best val_auc So Far: 0.7609689831733704
Total elapsed time: 01h 39m 13s


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [29]:
best_hps3.values


{'GRU Layer': 64,
 'Env Layer': 16,
 'Combined Layer': 16,
 'Env depth': 2,
 'Comb depth': 1,
 'learning_rate': 1e-05,
 'GRU activation': 'relu',
 'Env activation': 'tanh',
 'Comb activation': 'swish',
 'gru dropout ratio': 0.05,
 'env dropout ratio': 0.1,
 'comb dropout ratio': 0.15,
 'batch_size': 21}